In [1]:
import os
import cv2
import pickle
import numpy as np
from sklearn import svm
from skimage import io
from skimage.transform import resize
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
psvm = pickle.load(open('topmodel.pkl','rb'))
scaler = pickle.load(open('scaler.pkl','rb'))

In [3]:
### Resize and store in holdouts list

for file in os.listdir("holdouts/"):
    if file.endswith('.jpg'):
        path = "holdouts/" + file
        img = io.imread(path, as_grey=False)
        resized = resize(img, (32, 32))
        io.imsave(f'holdouts/{file[:-4]}.png', resized)

/Users/aaronfrederick/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/aaronfrederick/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [4]:
holdouts = []
filenames = []
for file in os.listdir("holdouts/"):
    if file.endswith('.png'):
        print(file)
        holdouts.append(np.asarray(cv2.imread(f"holdouts/{file}")))
        filenames.append(file)

aaronethan.png
deer.png
edel.png
scarjo.png
bird.png
pancake.png
sportscar.png
truck.png


In [5]:
reshaped = [np.reshape(matrix, (1,-1)) for matrix in holdouts]
reshaped = np.array(reshaped).reshape(len(reshaped),3072)
df = pd.DataFrame(reshaped)
df

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,211,252,255,215,253,253,216,254,254,216,...,137,21,0,127,48,12,207,53,11,212
1,160,159,175,157,151,168,164,182,183,153,...,98,179,192,194,139,177,164,121,211,163
2,96,62,47,92,62,47,90,61,46,91,...,21,26,20,20,27,20,20,29,21,21
3,185,193,162,185,193,162,185,193,162,187,...,71,97,90,127,49,51,86,53,50,86
4,73,58,56,75,60,58,66,54,51,62,...,110,87,91,102,76,79,84,95,96,96
5,132,145,147,131,144,152,150,163,171,142,...,184,208,215,218,198,205,208,180,187,190
6,28,31,29,25,39,40,29,37,37,38,...,88,98,98,98,78,78,78,31,34,39
7,221,166,91,220,163,87,218,162,83,219,...,175,99,172,194,0,137,141,78,168,185


In [6]:
imgs = scaler.transform(df)

In [10]:
imgs[0].shape

(3072,)

In [8]:
df['image_label'] = filenames
df['pred_label'] = psvm.predict(imgs)


In [9]:
df

,0,1,2,3,4,5,6,7,8,9,...,3064,3065,3066,3067,3068,3069,3070,3071,image_label,pred_label
0,211,252,255,215,253,253,216,254,254,216,...,0,127,48,12,207,53,11,212,aaronethan.png,1
1,160,159,175,157,151,168,164,182,183,153,...,192,194,139,177,164,121,211,163,deer.png,1
2,96,62,47,92,62,47,90,61,46,91,...,20,20,27,20,20,29,21,21,edel.png,1
3,185,193,162,185,193,162,185,193,162,187,...,90,127,49,51,86,53,50,86,scarjo.png,1
4,73,58,56,75,60,58,66,54,51,62,...,91,102,76,79,84,95,96,96,bird.png,0
5,132,145,147,131,144,152,150,163,171,142,...,215,218,198,205,208,180,187,190,pancake.png,1
6,28,31,29,25,39,40,29,37,37,38,...,98,98,78,78,78,31,34,39,sportscar.png,1
7,221,166,91,220,163,87,218,162,83,219,...,172,194,0,137,141,78,168,185,truck.png,1
